# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy.random import RandomState
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Регион 1

In [2]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
data_1[['id']].duplicated().sum()

10

In [5]:
data_1[data_1[['id']].duplicated()]

,id,f0,f1,f2,product
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
66136,74z30,1.084962,-0.312358,6.990771,127.643327
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
75715,Tdehs,0.112079,0.430296,3.218993,60.964018
90815,fiKDv,0.049883,0.841313,6.394613,137.346586
92341,TtcGQ,0.110711,1.022689,0.911381,101.318008
97785,bsk9y,0.378429,0.005837,0.160827,160.637302


In [6]:
data_1['id'].nunique()

99990

### Регион 2

In [7]:
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [8]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
data_2[data_2[['id']].duplicated()]

,id,f0,f1,f2,product
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


### Регион 3

In [10]:
data_3 = pd.read_csv('/datasets/geo_data_2.csv')
data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [11]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [12]:
data_3[data_3[['id']].duplicated()]

,id,f0,f1,f2,product
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


<b> В данных нет пропусков, все типы определены соответствующе.

## Обучение и проверка модели

В каждом регионе разобъем данные на выборки и обучим модель "линейная регрессия" для предсказания объема нефти. Проверим RMSE моделей.

Cоздадим функцию, которая по данным о регионе, разделит их на выборки, обучит модель и посчитает средние запасы нефти и rmse.

In [13]:
def prediction(data):
    features = data.drop(['id', 'product'], axis=1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    mean_product = predictions.mean()
    mse = mean_squared_error(target_valid, predictions)
    rmse = mse ** 0.5
    print('Средние запасы нефти:', mean_product)
    print('RMSE:', rmse)

### Регион 1

In [19]:
prediction(data_1)

Средние запасы нефти: 92.59256778438035
RMSE: 37.5794217150813


### Регион 2

In [25]:
prediction(data_2)

Средние запасы нефти: 68.728546895446
RMSE: 0.893099286775617


### Регион 3

In [31]:
prediction(data_3)

Средние запасы нефти: 94.96504596800489
RMSE: 40.02970873393434


<b> Самой точной оказалась модель, построенная на данных 2-го региона. Ее rmse близок к нулю. Модель 1, 3 показали наибольший средний продукт, но очень высокую погрешность.
   

## Подготовка к расчёту прибыли

Обозначим все данные.

In [32]:
points = 500
max_points = 200
budget = 10**10
price = 450000
risk = 2.5

Объем для безубыточности.

In [33]:
min_product = budget/max_points/price
min_product

111.11111111111111

Достаточно добывать 111.11 тысяч барелей с точки для безубыточности. Это больше, чем средние запасы в каждом регионе.

## Расчёт прибыли и рисков 

Напишем функцию, которая будет потсчитывать прибыль в регионе.

In [34]:
def calculating_profits(target, predictions, count_points, budget):
    best_points = pd.Series(predictions).sort_values(ascending=False)[:count_points]
    product_quantity = target.reset_index(drop=True)[best_points.index].sum()
    profit = price*product_quantity - budget
    return profit

Прибыль региона 1.

In [35]:
calculating_profits(target_valid_1, predictions_1, max_points, budget)

3320826043.1398506

Прибыль региона 2.

In [36]:
calculating_profits(target_valid_2, predictions_2, max_points, budget)

2415086696.681511

Прибыль региона 3.

In [37]:
calculating_profits(target_valid_3, predictions_3, max_points, budget)

2710349963.5998325

<b> Самую большую предполагаемую прибыль имеет регион 1. Но у модели 1 региона слишком большая погрешность, поэтому нельзя опираться на это значение.

Посчитаем все риски и прибыль. Применим Bootstrap для определения интервала прибыли и найдем ее среднее значение.

In [38]:
state = RandomState(12345)
def bootstrap_risc_profit(target, predictions):
    values = []
    for i in range(1000):
        subsample_for_target = pd.Series(target).reset_index(drop=True).sample(n = points, replace=True, random_state=state)
        subsample_for_predictions = predictions[subsample_for_target.index]
        profit = calculating_profits(subsample_for_target, subsample_for_predictions, max_points, budget)
        values.append(profit)
    values = pd.Series(values)
    confidence_interval = [values.quantile(0.025), values.quantile(0.975)]
    mean_profit = values.mean()
    risk = (((values < 0).sum())/ len(values)) * 100
    return mean_profit, risk, confidence_interval

Регион 1

In [39]:
mean_profit_1, risk_1, confidence_interval_1 = bootstrap_risc_profit(target_valid_1, predictions_1)
print('Средняя прибыль:', mean_profit_1)
print('Интервал доверия', confidence_interval_1)
print('Риск', round(risk_1, 1))

Средняя прибыль: 396164984.8023711
Интервал доверия [-111215545.89049526, 909766941.5534226]
Риск 6.9


Регион 2

In [40]:
mean_profit_2, risk_2, confidence_interval_2 = bootstrap_risc_profit(target_valid_2, predictions_2)
print('Средняя прибыль:', mean_profit_2)
print('Интервал доверия', confidence_interval_2)
print('Риск', round(risk_2, 1))

Средняя прибыль: 461155817.2772397
Интервал доверия [78050810.7517417, 862952060.2637234]
Риск 0.7


Регион 3

In [41]:
mean_profit_3, risk_3, confidence_interval_3 = bootstrap_risc_profit(target_valid_3, predictions_3)
print('Средняя прибыль:', mean_profit_3)
print('Интервал доверия', confidence_interval_3)
print('Риск', round(risk_3, 1))

Средняя прибыль: 392950475.17060447
Интервал доверия [-112227625.37857565, 934562914.5511636]
Риск 6.5


Регион 2 имеет самые маленькие риски и самую большую среднюю прибыль.

## Общий вывод 

<b> Выбираем регион 2, так как риски меньше 2.5%, у региона самый высокий показаетль средней прибыли и самая точная модель дла предсказаний (rmse близко к нулю).